In [51]:
import xmltodict, json
import requests
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv("ebay_api_key.env") #hide production key in environment file



APP_ID = os.getenv("EBAY_PROD_ID")


## URL 
HTTP call url to send the Production Client API. The Payload will modify the url below with a ? followed by an % and key value pairs. This is the most common and recommended way that ebay suggests users to send a request to the Finding API.

## HTTP Headers
Required headers to return a 200 response code. The top header is AuthnAuth security. As there is no need for a user token and the only interface is with my app, the only security required is for the app. The second row is the call be sent to the Finding API, findItemsAdvanced. ## THIS SHOULD BE A LINK DOG.   The third formats the respons as JSON.

## Payload
Modifies HTTP call. Search for knife in the Collectible-Modern-Factory-Manufactured-Folding-Knives that is of the Used Condition. This request returns 100 pages on page 1. 

In [71]:
url = "https://svcs.ebay.com/services/search/FindingService/v1" #url fir Finding API


headers = {
            "X-EBAY-SOA-SECURITY-APPNAME": APP_ID, #production client_id
            "X-EBAY-SOA-OPERATION-NAME": "findItemsAdvanced", #call_id
            "X-EBAY-API-RESPONSE-ENCODING": "JSON" #format respons as JSON
           }



payload = {
            "keywords":"knife", #knife in search bar
            "categoryId":"48818", #Collectible-Modern-Factory-Manufactured-Folding-Knives
            "itemFilter(0).name": "Condition",
            "itemFilter(0).value": "Used",
            "itemFilter(1).name": "ListingType",
            "itemFilter(1).value": "FixedPrice",
            "aspectFilter.aspectName(0)": "Brand",
            "aspectFilter.aspectValueName(0)": "Benchmade",
            "aspectFilter.aspectName(1)": "Brand",
            "aspectFilter.aspectValueName(1)": "Buck",
            "aspectFilter.aspectName(2)": "Brand",
            "aspectFilter.aspectValueName(2)": "Case",
            "aspectFilter.aspectName(3)": "Brand",
            "aspectFilter.aspectValueName(3)": "CRKT",
            "aspectFilter.aspectName(4)": "Brand",
            "aspectFilter.aspectValueName(4)": "Kershaw",
            "aspectFilter.aspectName(5)": "Brand",
            "aspectFilter.aspectValueName(5)": "Leatherman",
            "aspectFilter.aspectName(7)": "Brand",
            "aspectFilter.aspectValueName(7)": "Spyderco",
            "paginationInput.entriesPerPage": 100,
            "PaginationInput.pageNumber": 6
        
          }

In [72]:
r = requests.get(url, headers=headers, params=payload)
json_dict_100knivesp1 = xmltodict.parse(r.content)
knives_data_list_of_dicts = json_dict_100knivesp1['findItemsAdvancedResponse']['searchResult']['item']
total_pages = int(json_dict_100knivesp1['findItemsAdvancedResponse']['paginationOutput']['totalPages'])
total_pages

57

In [65]:
#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['itemId', 'title', 'galleryURL', 
        'viewItemURL', 'postalCode', 'location', 
        'country', 'shippingInfo', 'sellingStatus', 
        'listingInfo', 'returnsAccepted', 'condition']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
    
       
        # Add to list if all values are present
        if all(prepared_data.values()):
            results.append(prepared_data)
    
    return results

In [66]:
#organize call with function
prepared_knives = prepare_data(knives_data_list_of_dicts)

In [67]:
total_pages

57

In [69]:
# Create an empty list for the full prepared dataset
full_dataset = []

for page in range(1, total_pages):
    # Add or update the "offset" key-value pair in url_params
    payload["paginationInput.pageNumber"] = page
    
    # Make the query and get the response
    response = requests.get(url, headers=headers, params=payload)
    
    # Get the response body in JSON format
    response_json = xmltodict.parse(r.content)
    
    # Get the list of businesses from the response_json
    knives_data_list_of_dicts = response_json['findItemsAdvancedResponse']['searchResult']['item']
    
    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(knives_data_list_of_dicts)
    
    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df = pd.DataFrame(full_dataset)

5544

In [70]:
full_dataset

[{'itemId': '134165340692',
  'title': 'CRKT Fossil Folding Knife',
  'galleryURL': 'https://i.ebayimg.com/thumbs/images/g/uKkAAOSwFRxixPCs/s-l140.jpg',
  'viewItemURL': 'https://www.ebay.com/itm/CRKT-Fossil-Folding-Knife-/134165340692',
  'postalCode': '786**',
  'location': 'Kyle,TX,USA',
  'country': 'US',
  'shippingInfo': {'shippingServiceCost': {'@currencyId': 'USD',
    '#text': '2.99'},
   'shippingType': 'Flat',
   'shipToLocations': 'Worldwide',
   'expeditedShipping': 'false',
   'oneDayShippingAvailable': 'false',
   'handlingTime': '2'},
  'sellingStatus': {'currentPrice': {'@currencyId': 'USD', '#text': '22.99'},
   'convertedCurrentPrice': {'@currencyId': 'USD', '#text': '22.99'},
   'sellingState': 'Active',
   'timeLeft': 'P28DT9H36M26S'},
  'listingInfo': {'bestOfferEnabled': 'false',
   'buyItNowAvailable': 'false',
   'startTime': '2022-07-06T02:19:50.000Z',
   'endTime': '2022-08-06T02:19:50.000Z',
   'listingType': 'FixedPrice',
   'gift': 'false',
   'watchCount'

In [14]:
# df.to_csv('data/full_dataset.csv', index=False)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5650 entries, 0 to 5649
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   itemId           5650 non-null   object
 1   title            5650 non-null   object
 2   galleryURL       5650 non-null   object
 3   viewItemURL      5650 non-null   object
 4   postalCode       5650 non-null   object
 5   location         5650 non-null   object
 6   country          5650 non-null   object
 7   shippingInfo     5650 non-null   object
 8   sellingStatus    5650 non-null   object
 9   listingInfo      5650 non-null   object
 10  returnsAccepted  5650 non-null   object
 11  condition        5650 non-null   object
dtypes: object(12)
memory usage: 529.8+ KB


In [47]:
df = df.drop_duplicates(subset=['title', 'galleryURL'], keep='first')

In [49]:
import re

In [50]:
#Create row for converted Price of Knives in US dollars
price_list = []
for row in full_dataset:
    listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['#text'])
    price_list.append(listed_price)
    
df['price_in_US'] = price_list

ValueError: Length of values (5650) does not match length of index (50)

In [18]:
df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))

In [19]:
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))

In [20]:
df.drop(df[df['shipping_cost'] == ''].index, inplace=True)

In [21]:
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

In [22]:
df['shipping_cost']

0        3.50
1        0.00
2        2.99
3        0.00
4        0.00
         ... 
10390    0.00
10391    5.25
10392    0.00
10393    3.99
10394    0.00
Name: shipping_cost, Length: 10185, dtype: float64

In [23]:
df['price_in_US']

0         7.25
1         8.53
2        22.99
3         9.95
4        65.00
         ...  
10390    15.17
10391    59.99
10392    36.00
10393    16.00
10394    28.99
Name: price_in_US, Length: 10185, dtype: float64

In [24]:
df['converted_price'] = df['shipping_cost'] + df['price_in_US']

In [25]:
df['converted_price']

0        10.75
1         8.53
2        25.98
3         9.95
4        65.00
         ...  
10390    15.17
10391    65.24
10392    36.00
10393    19.99
10394    28.99
Name: converted_price, Length: 10185, dtype: float64

In [26]:
df.sample(20)

,itemId,title,galleryURL,viewItemURL,postalCode,location,country,shippingInfo,sellingStatus,listingInfo,returnsAccepted,condition,price_in_US,shipping_cost,converted_price
5574,144626838786,Kershaw Showtime 1955 Pocket Knife,https://i.ebayimg.com/thumbs/images/g/zYAAAOSw...,https://www.ebay.com/itm/Kershaw-Showtime-1955...,472**,"North Vernon,IN,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",21.00,0.00,21.00
4989,363897341967,Victorinox Classic Swiss Army Knife with Logo ...,https://i.ebayimg.com/thumbs/images/g/oscAAOSw...,https://www.ebay.com/itm/Victorinox-Classic-Sw...,277**,"Durham,NC,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",8.50,3.50,12.00
4398,204030012695,"Victorinox Swiss Army Rambler Pocket Knife, Re...",https://i.ebayimg.com/thumbs/images/g/n0EAAOSw...,https://www.ebay.com/itm/Victorinox-Swiss-Army...,492**,"Jackson,MI,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",13.95,0.00,13.95
9224,225062041690,Benchmade USA 484 Nakamura M390 Pocket knife,https://i.ebayimg.com/thumbs/images/g/8y0AAOSw...,https://www.ebay.com/itm/Benchmade-USA-484-Nak...,242**,"Abingdon,VA,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",209.00,9.99,218.99
8911,233805693414,Victorinox Classic SD Swiss Army Knife 3 Tool ...,https://i.ebayimg.com/thumbs/images/g/3f4AAOSw...,https://www.ebay.com/itm/Victorinox-Classic-SD...,303**,"Atlanta,GA,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",8.53,0.00,8.53
8466,234610472528,Benchmade Griptilian 550 Pardue Design Axis Sh...,https://i.ebayimg.com/thumbs/images/g/SZYAAOSw...,https://www.ebay.com/itm/Benchmade-Griptilian-...,782**,"San Antonio,TX,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",false,"{'conditionId': '3000', 'conditionDisplayName'...",89.00,0.00,89.00
9741,363897341967,Victorinox Classic Swiss Army Knife with Logo ...,https://i.ebayimg.com/thumbs/images/g/oscAAOSw...,https://www.ebay.com/itm/Victorinox-Classic-Sw...,277**,"Durham,NC,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",8.50,3.50,12.00
4904,134166975679,Vtg Victorinox CLASSIC SD Swiss Army Knife BLA...,https://i.ebayimg.com/thumbs/images/g/QGgAAOSw...,https://www.ebay.com/itm/Vtg-Victorinox-CLASSI...,120**,"Duanesburg,NY,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",false,"{'conditionId': '3000', 'conditionDisplayName'...",14.95,5.00,19.95
3100,265745003534,Benchmade 551 Griptilian Folding Blade Pocket ...,https://i.ebayimg.com/thumbs/images/g/1UsAAOSw...,https://www.ebay.com/itm/Benchmade-551-Griptil...,350**,"Gardendale,AL,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",72.95,0.00,72.95
3661,25561941843

In [27]:
df.describe()

,price_in_US,shipping_cost,converted_price
count,10185.000000,10185.000000,10185.000000
mean,43.250000,1.388247,44.638247
std,67.888385,2.604868,68.237249
min,2.800000,0.000000,6.600000
25%,15.170000,0.000000,16.180000
50%,22.990000,0.000000,23.740000
75%,38.000000,2.990000,38.000000
max,569.000000,10.000000,569.000000


In [28]:
df.to_csv('data/full_dataset.csv', index=False)

In [33]:
df = df.drop_duplicates(subset=['title', 'galleryURL'], keep='first')

In [34]:
df.to_csv('data/full_dataset4.csv', index=False)

In [35]:
df.describe()

,price_in_US,shipping_cost,converted_price
count,97.000000,97.000000,97.000000
mean,43.250000,1.388247,44.638247
std,68.237704,2.618272,68.588363
min,2.800000,0.000000,6.600000
25%,15.170000,0.000000,16.180000
50%,22.990000,0.000000,23.740000
75%,38.000000,2.990000,38.000000
max,569.000000,10.000000,569.000000


In [ ]:
# # Make a new list to hold results
#     results = []
    
#     for business_data in full_dataset:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['itemId', 'title', 'galleryURL', 
#         'viewItemURL', 'postalCode', 'location', 
#         'country', 'shippingInfo', 'sellingStatus', 
#         'listingInfo', 'returnsAccepted', 'condition']
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
    
       
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
#     return results

In [ ]:
# #create function for organizing API call
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['shippingServiceCost]
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
    
       
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    return results